# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
import pandas as pd 
import numpy as np
import re
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score
from scipy.stats.mstats import gmean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaterrespond.db')
df = pd.read_sql_table('Messages',engine)
X = df['message']
Y = df.drop(['message','genre','original','id'],axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    clean_words = []
    for word in words: 
        clean_word = lemmatizer.lemmatize(word).lower().strip()
        clean_words.append(clean_word)
    return clean_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [16]:
category_names = Y.columns
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))


------------------------------------------------------------ 
 Category: related 
              precision    recall  f1-score   support

          0       0.60      0.50      0.55      1518
          1       0.86      0.90      0.88      4990
          2       0.24      0.37      0.29        46

avg / total       0.79      0.80      0.80      6554

Accuracy of related 0.801495270064
------------------------------------------------------------ 
 Category: request 
              precision    recall  f1-score   support

          0       0.90      0.97      0.94      5457
          1       0.78      0.47      0.59      1097

avg / total       0.88      0.89      0.88      6554

Accuracy of request 0.889533109551
------------------------------------------------------------ 
 Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
def multioutput_f1score(y_true, y_pred): 
    scores = []
    for i in range(Y.shape[1]): 
        score = f1_score(y_true.iloc[:,i], y_pred[:,i],average='weighted')
        scores.append(score)
    scores = np.asarray(scores)
    scores = gmean(scores)
    return scores

In [37]:
multioutput_f1score(y_test,y_pred)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.93034539820270123

In [ ]:
for col in range(Y.shape[1]): 
    f1_score(y_test.iloc[:,0], y_pred.iloc[:,0])    

In [24]:
y_pred[:,0]

(6554,)

In [26]:
f1_score(y_test.iloc[:,0], y_pred[:,0],average='weighted')  

0.79602314771343141

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.